In [1]:
import hopsworks
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import datetime
import os

In [2]:
# Uncomment this cell and fill in details if you are running your own Hopsworks cluster
# key=""
# with open("api-key.txt", "r") as f:
#     key = f.read().rstrip()
# os.environ['HOPSWORKS_PROJECT']="cjsurf"
# os.environ['HOPSWORKS_HOST']="35.187.178.84"
# os.environ['HOPSWORKS_API_KEY']=key

In [3]:
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/135
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
mr = project.get_model_registry()
model = mr.get_model("cjsurf_model", version=3)
model_dir = model.download()

Connected. Call `.close()` to terminate connection gracefully.


In [5]:
import joblib
model = joblib.load(model_dir + "/knn_model.pkl")

In [6]:
fv = fs.get_feature_view("lahinch_surf", version=1)
fv.init_serving(1)

In [7]:
key = {"beach_id" : 1}
vector = fv.get_feature_vector(key)
print(vector)

[0.97, 8.4, 145, datetime.datetime(2022, 7, 4, 4, 0), 0.95, 8.3, 144.0, 1.6569504e+18, 0.97, 8.2, 144.0, 1.6569576e+18, 1.19, 7.6, 125.0, 1.6569648e+18, 0.89, 7.8, 102.0, 1.656972e+18, 0.9, 7.8, 104.0, 1.6569792e+18, 0.91, 7.8, 105.0, 1.6569864e+18, 0.92, 7.6, 105.0, 1.6569936e+18, 0.89, 6.9, 155.0, 1.6570008e+18, 0.88, 6.8, 156.0, 1.657008e+18, 1.15, 9.0, 153.0, 1.6570152e+18, 1.21, 9.0, 153.0, 1.6570224e+18, 1.27, 9.1, 153.0, 1.6570296e+18, 1.31, 9.1, 153.0, 1.6570368e+18, 1.34, 9.1, 152.0, 1.657044e+18, 1.35, 9.1, 151.0, 1.6570512e+18, 1.36, 8.9, 150.0, 1.6570584e+18, 1.36, 8.8, 149.0, 1.6570656e+18, 1.34, 8.8, 147.0, 1.6570728e+18, 1.28, 8.8, 149.0, 1.65708e+18, 1.26, 8.5, 148.0, 1.6570872e+18, 1.41, 8.3, 135.0, 1.6570944e+18, 1.4, 8.2, 136.0, 1.6571016e+18, 1.39, 8.1, 136.0, 1.6571088e+18, 1.39, 8.0, 136.0, 1.657116e+18, 1.4, 8.0, 138.0, 1.6571232e+18, 1.39, 7.9, 139.0, 1.6571304e+18, 1.38, 7.8, 140.0, 1.6571376e+18, 1.36, 7.8, 140.0, 1.6571448e+18, 1.34, 7.7, 140.0, 1.657152e+18,

In [8]:
feature_vectors = []
# Remove the 'pred_dtime' column
feature_vectors.append(vector[0:3])
for i in range(4,len(vector),4):
    feature_vectors.append(vector[i:i+3])
# 'pred_dtime'
dt = vector[3]
feature_vectors

[[0.97, 8.4, 145],
 [0.95, 8.3, 144.0],
 [0.97, 8.2, 144.0],
 [1.19, 7.6, 125.0],
 [0.89, 7.8, 102.0],
 [0.9, 7.8, 104.0],
 [0.91, 7.8, 105.0],
 [0.92, 7.6, 105.0],
 [0.89, 6.9, 155.0],
 [0.88, 6.8, 156.0],
 [1.15, 9.0, 153.0],
 [1.21, 9.0, 153.0],
 [1.27, 9.1, 153.0],
 [1.31, 9.1, 153.0],
 [1.34, 9.1, 152.0],
 [1.35, 9.1, 151.0],
 [1.36, 8.9, 150.0],
 [1.36, 8.8, 149.0],
 [1.34, 8.8, 147.0],
 [1.28, 8.8, 149.0],
 [1.26, 8.5, 148.0],
 [1.41, 8.3, 135.0],
 [1.4, 8.2, 136.0],
 [1.39, 8.1, 136.0],
 [1.39, 8.0, 136.0],
 [1.4, 8.0, 138.0],
 [1.39, 7.9, 139.0],
 [1.38, 7.8, 140.0],
 [1.36, 7.8, 140.0],
 [1.34, 7.7, 140.0],
 [1.33, 7.7, 140.0],
 [1.32, 7.7, 141.0],
 [1.28, 7.7, 141.0],
 [1.25, 7.7, 141.0],
 [1.23, 7.7, 141.0],
 [1.21, 7.7, 141.0],
 [1.22, 7.7, 141.0],
 [1.19, 7.5, 140.0],
 [1.04, 7.2, 134.0],
 [1.06, 7.4, 136.0],
 [1.04, 7.4, 135.0],
 [0.96, 8.8, 172.0],
 [0.95, 6.9, 131.0],
 [0.86, 8.6, 173.0],
 [0.83, 8.5, 173.0],
 [0.79, 8.4, 173.0],
 [0.8, 7.0, 119.0],
 [0.77, 7.3, 118.0]

In [9]:
row_list=[]
columns=["beach_id","hits_at", "wave_height"]
for i in range(0,len(feature_vectors)):
    arr=[]
    arr.append(feature_vectors[i])
    res = model.predict(arr)
    row = []
    row.append(1) #beach_id
    hour = datetime.timedelta(0, (1*3600*i))
    ts = dt + hour
    dt_str = ts.strftime("%Y-%m-%d %H:%M") #:%S
    row.append(dt_str)
    row.append(res[0])
    row_list.append(row)

row_list

[[1, '2022-07-04 04:00', 3],
 [1, '2022-07-04 05:00', 1],
 [1, '2022-07-04 06:00', 1],
 [1, '2022-07-04 07:00', 3],
 [1, '2022-07-04 08:00', 0],
 [1, '2022-07-04 09:00', 3],
 [1, '2022-07-04 10:00', 3],
 [1, '2022-07-04 11:00', 3],
 [1, '2022-07-04 12:00', 3],
 [1, '2022-07-04 13:00', 3],
 [1, '2022-07-04 14:00', 3],
 [1, '2022-07-04 15:00', 3],
 [1, '2022-07-04 16:00', 3],
 [1, '2022-07-04 17:00', 3],
 [1, '2022-07-04 18:00', 3],
 [1, '2022-07-04 19:00', 3],
 [1, '2022-07-04 20:00', 3],
 [1, '2022-07-04 21:00', 3],
 [1, '2022-07-04 22:00', 3],
 [1, '2022-07-04 23:00', 3],
 [1, '2022-07-05 00:00', 3],
 [1, '2022-07-05 01:00', 0],
 [1, '2022-07-05 02:00', 1],
 [1, '2022-07-05 03:00', 1],
 [1, '2022-07-05 04:00', 1],
 [1, '2022-07-05 05:00', 1],
 [1, '2022-07-05 06:00', 1],
 [1, '2022-07-05 07:00', 3],
 [1, '2022-07-05 08:00', 3],
 [1, '2022-07-05 09:00', 3],
 [1, '2022-07-05 10:00', 3],
 [1, '2022-07-05 11:00', 3],
 [1, '2022-07-05 12:00', 3],
 [1, '2022-07-05 13:00', 3],
 [1, '2022-07-

In [10]:
import pandas as pd

df = pd.DataFrame(row_list, columns=columns)

df

,beach_id,hits_at,wave_height
0,1,2022-07-04 04:00,3
1,1,2022-07-04 05:00,1
2,1,2022-07-04 06:00,1
3,1,2022-07-04 07:00,3
4,1,2022-07-04 08:00,0
...,...,...,...
114,1,2022-07-08 22:00,1
115,1,2022-07-08 23:00,5
116,1,2022-07-09 00:00,5
117,1,2022-07-09 01:00,1


In [ ]:
fs = project.get_feature_store()

preds_fg = fs.get_or_create_feature_group(name="wave_predictions",
                version=1,
                primary_key=["beach_id", "hits_at"],
                description="Lahinch surf height predictions",
                online_enabled=True
                )
preds_fg.insert(df)   

In [ ]:
import plotly.express as px
fig = px.line(df, x = "hits_at", y = "wave_height", 
              #markers=True, 
              title = "Wave Heights at Lahinch"
             )
fig.update_layout(
#    plot_bgcolor="white",
    margin=dict(t=50,l=10,b=10,r=10)
)
fig.update_layout(
    xaxis_tickformat = '%d/%m (%a)<br>Time %h:%m <br> %Y'
)
fig.update_xaxes(
#        tickangle = 90,
#        nticks=20,
        ticks="outside", 
        tickwidth=2,
        tickcolor='black',
        ticklen=10,
        title_text = "Time",
        title_font = {"size": 20},
        title_standoff = 25)
fig.update_yaxes(
        title_text = "Wave Height (ft)",
        title_font = {"size": 20},
        title_standoff = 25)

fig.show()


In [ ]:
filename="latest_lahinch.png"
fig.write_image(file=filename, format="png", width=1920, height=1280)
dataset_api = project.get_dataset_api()
uploaded_file_path = dataset_api.upload(filename, "Resources", overwrite=True)

# update images with predictions of previous week and outcomes